In [1]:
# install SNN-Toolbox
!pip install /content/drive/My\ Drive/GitHub/spiking-p3-gtnumbers/snn_toolbox

Processing ./drive/My Drive/GitHub/spiking-p3-gtnumbers/snn_toolbox
  Created wheel for snntoolbox: filename=snntoolbox-0.4.1-py2.py3-none-any.whl size=195127 sha256=b55056098ac3b8c5b6021f0775932f8ff6d4c51ff2b9a5881463ec12d3fa8400
  Stored in directory: /tmp/pip-ephem-wheel-cache-e2ro3869/wheels/a6/22/3b/c7bbca8a559a101b4cfb0bef50ae45a6e62a1cf48ae44ffcb8
Successfully built snntoolbox


In [2]:
%matplotlib inline

from scipy.io import loadmat
import numpy as np
import keras
import logging
from keras.layers import (Conv2D, BatchNormalization, Dropout, AveragePooling2D,
Flatten, Dense, Input)
from keras.models import Sequential
from sklearn.model_selection import cross_val_score, ShuffleSplit
import tensorflow as tf
import os
import datetime
import snntoolbox
from snntoolbox.utils.utils import import_configparser
from snntoolbox.bin.run import main
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
# set logging level
logging.basicConfig(level=logging.INFO)
# for replication
np.random.seed(1)
tf.random.set_seed(2)
# set up working directory to store results
path_wd = os.path.abspath(
    os.path.join('/content/drive/My Drive/', 'temp', 'gtn',
                 str(datetime.datetime.now())))
os.makedirs(path_wd)
model_name = 'gtn_model'
model_ext = '.h5'
# load data
datapath = '/content/drive/My Drive/VarekaGTNEpochs.mat'
data = loadmat(datapath)
target = data['allTargetData']
non_target = data['allNonTargetData']
logging.info('Loaded data from file {}.'.format(datapath))

INFO:root:Loaded data from file /content/drive/My Drive/VarekaGTNEpochs.mat.


In [4]:
def discard_damaged(features, labels):
  """
  preprocess data
  """
  threshold = 100.0
  accept_features = []
  accept_labels = []
  for i in range(features.shape[0]):
      discard = False
      # check max values of all channels
      if np.max(np.abs(features[i])) > threshold:
          discard = True
      if not discard:
          accept_features.append(features[i])
          accept_labels.append(labels[i])
  logging.info('Discarded {}% of samples.'.format(
      (1 - len(accept_features) / features.shape[0]) * 100))
  features = np.array(accept_features)
  labels = np.array(accept_labels)
  return features, labels

features = np.concatenate((target, non_target), axis=0)
target_labels = np.tile(np.array([1, 0]), (target.shape[0], 1))
non_target_labels = np.tile(np.array([0, 1]), (non_target.shape[0], 1))
labels = np.vstack((target_labels, non_target_labels))
(features, labels) = discard_damaged(features, labels)
features = np.expand_dims(features, -1)
logging.debug('preprocessed features({}): {})'.format(features.shape, features))
logging.debug('preprocessed labels({}): {})'.format(labels.shape, labels))

INFO:root:Discarded 30.315643426985783% of samples.


In [0]:
def create_base_config():
  """
  Create a config file for SNN Toolbox.
  """
  configparser = import_configparser()
  config = configparser.ConfigParser()
  config['paths'] = {
      'path_wd': path_wd,  # Path to model.
      'dataset_path': path_wd,  # Path to dataset.
      'filename_ann': model_name  # Name of input model.
  }

  config['tools'] = {
      'evaluate_ann': False,  # Test ANN on dataset before conversion.
      'normalize': True  # Normalize weights for full dynamic range.
  }

  config['simulation'] = {
      'simulator': 'INI',  # Chooses execution backend of SNN toolbox.
      'duration': 50,  # Number of time steps to run each sample.
      'num_to_test': 2009,  # How many test samples to run.
      'batch_size': 49,  # Batch size for simulation.
  }

  # config['output'] = {
  #     'plot_vars': {
  #         'spiketrains',
  #         'spikerates',
  #         'activations',
  #         'correlation',
  #         'v_mem',
  #         'error_t'}
  # }
  return config


def write_config(config_name='config', config=None):
  """
  Store config file.
  """
  config_path = os.path.join(path_wd, config_name)
  with open(config_path, 'w') as configfile:
      config.write(configfile)
  logging.info('SNN-Toolbox config file written to {}'.format(config_path))
  return config_path

In [0]:
def create_model():
  """
  Prepare Keras model
  """
  model = Sequential()
  model.add(Conv2D(filters=6, kernel_size=(3, 3),
    activation=keras.activations.elu, input_shape=(3, 1200, 1)))
  model.add(BatchNormalization())
  model.add(Dropout(0.5, seed=0))
  model.add(AveragePooling2D(pool_size=(1, 8)))
  model.add(Flatten())
  model.add(Dense(100, activation=keras.activations.elu))
  model.add(BatchNormalization())
  model.add(Dropout(0.5, seed=0))
  model.add(Dense(2, activation=keras.activations.softmax))
  model.compile(
      optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(),
      # metrics=['accuracy', keras.metrics.AUC(), keras.metrics.Recall(),
      #         keras.metrics.Precision()])
      metrics=['accuracy'])
  return model

In [0]:
config = create_base_config()
config_path = write_config(config_name='config', config=config)

x_train, x_test, y_train, y_test = train_test_split(
            features, labels, test_size=0.25, random_state=0, shuffle=True
        )
# save test data for SNN Toolbox
logging.debug('Saving test data...')
np.savez_compressed(os.path.join(path_wd, 'x_test'), x_test)
np.savez_compressed(os.path.join(path_wd, 'y_test'), y_test)

shuffle_split = ShuffleSplit(
    n_splits=30,
    test_size=0.25,
    random_state=0
)
es = keras.callbacks.callbacks.EarlyStopping(
    patience=5, verbose=1, restore_best_weights=True)
counter = 0
val_metrics = []
test_metrics = []
snn_metrics = []
for train, validation in shuffle_split.split(x_train):
  logging.info('Iteration {}'.format(counter))
  ann_model = create_model()
  val_metrics.append(
      ann_model.fit(
        x=x_train[train], 
        y=y_train[train],
        callbacks=[es],
        shuffle=True,
        epochs=30,
        batch_size=16,
        validation_data=(x_train[validation], y_train[validation])
    ))
  ann_res = ann_model.evaluate(x_test, y_test)
  test_metrics.append(ann_res)
  print('test accuracy: {}'.format(ann_res))
  logging.info('Saving ANN model...')
  ann_model.save(os.path.join(
      path_wd, ''.join([model_name, model_ext])))
  np.savez_compressed(os.path.join(path_wd, 'x_norm'), (x_train[train])[::10])
  logging.info('Converting ANN to SNN...')
  snn_metrics.append(main(config_path))
  counter += 1
  logging.debug('end of iteration {}...'.format(counter))
logging.info('Saving results to {}'.format(path_wd))
np.savez_compressed(os.path.join(path_wd, 'val_metrics'), np.array(val_metrics))
np.savez_compressed(os.path.join(path_wd, 'ann_metrics'), np.array(test_metrics))
np.savez_compressed(os.path.join(path_wd, 'snn_metrics'), snn_metrics)
logging.debug('Finished')

INFO:root:SNN-Toolbox config file written to /content/drive/My Drive/temp/gtn/2020-04-30 15:05:49.615516/config
INFO:root:Iteration 0


Train on 4520 samples, validate on 1507 samples
Epoch 1/30
4520/4520 [==============================] - 6s 1ms/step - loss: 0.9548 - accuracy: 0.5544 - val_loss: 0.6934 - val_accuracy: 0.5985
Epoch 2/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.7328 - accuracy: 0.5838 - val_loss: 0.6668 - val_accuracy: 0.6025
Epoch 3/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6727 - accuracy: 0.6080 - val_loss: 0.6579 - val_accuracy: 0.6231
Epoch 4/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6522 - accuracy: 0.6221 - val_loss: 0.6518 - val_accuracy: 0.6357
Epoch 5/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6425 - accuracy: 0.6301 - val_loss: 0.6583 - val_accuracy: 0.6191
Epoch 6/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6401 - accuracy: 0.6352 - val_loss: 0.6652 - val_accuracy: 0.6105
Epoch 7/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6346 - acc

INFO:root:Saving ANN model...


test accuracy: [0.6379722675736001, 0.6475858688354492]


INFO:root:Converting ANN to SNN...


...

Simulation finished.


Total accuracy: 55.65% on 2009 test samples.


Accuracy averaged by class size: 55.66%
Train on 4520 samples, validate on 1507 samples
Epoch 1/30
4520/4520 [==============================] - 6s 1ms/step - loss: 0.9324 - accuracy: 0.5509 - val_loss: 0.7109 - val_accuracy: 0.6032
Epoch 2/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.7303 - accuracy: 0.5936 - val_loss: 0.6778 - val_accuracy: 0.6078
Epoch 3/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6697 - accuracy: 0.6168 - val_loss: 0.6641 - val_accuracy: 0.6204
Epoch 4/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6505 - accuracy: 0.6283 - val_loss: 0.6542 - val_accuracy: 0.6291
Epoch 5/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6385 - accuracy: 0.6365 - val_loss: 0.6592 - val_accuracy: 0.6171
Epoch 6/30
4520/4520 [==============================] - 5s 1ms/step - loss: 0.6444 - accuracy: 0.6294 - val_loss: 0.6588 -

INFO:root:Saving ANN model...


test accuracy: [0.6450728630770374, 0.6371328830718994]


INFO:root:Converting ANN to SNN...


Initializing INI simulator...

Loading scale factors from disk instead of recalculating.
Loading data set from '.npz' files in /content/drive/My Drive/temp/gtn/2020-04-30 15:05:49.615516.

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 1, 1198, 6)        60        
_________________________________________________________________
batch_normalization_59 (Batc (None, 1, 1198, 6)        24        
_________________________________________________________________
dropout_59 (Dropout)         (None, 1, 1198, 6)        0         
_________________________________________________________________
average_pooling2d_30 (Averag (None, 1, 149, 6)         0         
_________________________________________________________________
flatten_30 (Flatten)         (None, 894)               0         
______________________________________________________________

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# test_metrics = np.load('/content/drive/My Drive/temp/2020-04-28 09:57:25.354714/ann_metrics.npz')['arr_0'][:, 1:]
# snn_metrics = np.load('/content/drive/My Drive/temp/2020-04-28 09:57:25.354714/snn_metrics.npz')['arr_0']

metrics = np.hstack((test_metrics, snn_metrics))
metrics_df = pd.DataFrame(metrics, columns=['ANN', 'SNN'])

ann_mean = metrics_df['ANN'].mean(axis=0)
snn_mean = metrics_df['SNN'].mean(axis=0)

ann_std = metrics_df['ANN'].std(axis=0)
snn_std = metrics_df['SNN'].std(axis=0)

characteristics = pd.DataFrame(
    {'ANN':{'mean':ann_mean, 'std':ann_std},
     'SNN':{'mean':snn_mean, 'std':snn_std}})

print(characteristics)

metrics_df.plot(kind='bar')